In [11]:
import pandas as pd
import numpy as np
import cv2



In [14]:
train_data = pd.read_csv("datasets/csv/train_data.csv")

def preprocess_image(img_path):
    # reading images in grayscale
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # target_size of 224, 224 commonly used for image classification
    img = cv2.resize(img, (224, 224))
    # normalising pixel values
    img_array = img.astype(np.float32) / 255
    return img_array

# using preprocessed images as data
X_train = np.array([preprocess_image(image_path) for image_path in train_data["image_path"]])
# using "pathology" column as labels
y_train = np.array(train_data["pathology"])

In [9]:
test_data = pd.read_csv("datasets/csv/test_data.csv")